In [1]:
try: import fastkaggle
except ModuleNotFoundError:
    !pip install -Uq fastkaggle

from fastkaggle import *
print("fastkaggle version: ", fastkaggle.__version__)

0.0.7


In [2]:
comp = 'titanic'
path = setup_comp(comp, install = 'fastai "timm >= 0.6.2.dev0"')

In [3]:
try: import fastai
except ModuleNotFoundError:
    !pip install -Uq fastai

from fastai.imports import *
print("fastai version: ", fastai.__version__)

2.7.4


In [4]:
!ls {path}

gender_submission.csv  test.csv  train.csv


In [5]:
df = pd.read_csv(path/'train.csv', index_col = 'PassengerId')
df.head()

,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
PassengerId,,,,,,,,,,,
1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S
2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Thayer)",female,38.0,1,0,PC 17599,71.2833,C85,C
3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S
4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S
5,0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S


In [6]:
y_train = df['Survived']
X_train = df.drop(['Survived', 'Name'], axis = 1)

In [7]:
from fastai.tabular.all import *

In [8]:
def to_cat(df = df):
    '''
    Convert string-type columns from a dataframe into categorical columns
    '''
    # Identify string/categorical columns in the dataframe
    _, cat = cont_cat_split(df, 1)
    
    # Convert to categorical type using for loops
    for col in cat:
        df[col] = pd.Categorical(df[col])

In [9]:
to_cat(X_train)

In [10]:
X_train.head()

,Pclass,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
PassengerId,,,,,,,,,
1,3,male,22.0,1,0,A/5 21171,7.2500,NaN,S
2,1,female,38.0,1,0,PC 17599,71.2833,C85,C
3,3,female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S
4,1,female,35.0,1,0,113803,53.1000,C123,S
5,3,male,35.0,0,0,373450,8.0500,NaN,S


In [12]:
try: from autoPyTorch.api.tabular_classification import TabularClassificationTask
except ModuleNotFoundError:
    !pip install -Uq autoPyTorch

from autoPyTorch.api.tabular_classification import TabularClassificationTask
import autoPyTorch
print("autoPyTorch version: ", autoPyTorch.__version__)

autoPyTorch version:  0.2


If this error occurs:
> ImportError: libGL.so.1: cannot open shared object file: No such file or directory

Run the following two commands in the shell:
```bash
sudo apt update
sudo apt install libgl1-mesa-glx
```

In [13]:
# initialise Auto-PyTorch api
api = TabularClassificationTask()

# Search for an ensemble of machine learning algorithms
api.search(
    X_train=X_train,
    y_train=y_train,
    optimize_metric='accuracy',
    total_walltime_limit=300,
    func_eval_time_limit_secs=50,
    memory_limit = None
)

[ERROR] [2022-08-23 17:14:35,181:Client-AutoPyTorch:ee0c884e-2306-11ed-af97-b6f6766cb37b:1] Traditional prediction for lgb failed with run state StatusType.TIMEOUT.
Additional info:
error: Timeout
configuration_origin: traditional
[ERROR] [2022-08-23 17:14:48,321:Client-AutoPyTorch:ee0c884e-2306-11ed-af97-b6f6766cb37b:1] Traditional prediction for knn failed with run state StatusType.CRASHED.
Additional info:
traceback: Traceback (most recent call last):
  File "/root/mambaforge/lib/python3.9/site-packages/autoPyTorch/evaluation/tae.py", line 61, in fit_predict_try_except_decorator
    ta(queue=queue, **kwargs)
  File "/root/mambaforge/lib/python3.9/site-packages/autoPyTorch/evaluation/train_evaluator.py", line 512, in eval_train_function
    evaluator.fit_predict_and_loss()
  File "/root/mambaforge/lib/python3.9/site-packages/autoPyTorch/evaluation/train_evaluator.py", line 186, in fit_predict_and_loss
    y_train_pred, y_opt_pred, y_valid_pred, y_test_pred = self._fit_and_predict(pip

/root/mambaforge/lib/python3.9/site-packages/smac/intensification/parallel_scheduling.py:154: UserWarning: Hyperband is executed with 1 workers only. Consider to use pynisher to use all available workers.
  warnings.warn(


In [14]:
print(api.sprint_statistics())

KeyError: 'test_loss'

In [15]:
print(api.leaderboard())

AttributeError: 'TabularClassificationTask' object has no attribute 'leaderboard'

In [16]:
df_test = pd.read_csv(path/'test.csv', index_col = 'PassengerId')
df_test = df_test.drop('Name', axis = 1)
to_cat(df_test)
df_test.head()

,Pclass,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
PassengerId,,,,,,,,,
892,3,male,34.5,0,0,330911,7.8292,NaN,Q
893,3,female,47.0,1,0,363272,7.0000,NaN,S
894,2,male,62.0,0,0,240276,9.6875,NaN,Q
895,3,male,27.0,0,0,315154,8.6625,NaN,S
896,3,female,22.0,1,1,3101298,12.2875,NaN,S


In [17]:
prediction = api.predict(df_test)

In [18]:
subm = pd.DataFrame(prediction, index = df_test.index, columns = ['Survived'])

In [19]:
subm.to_csv('subm.csv')
!head subm.csv

PassengerId,Survived
892,0
893,1
894,0
895,1
896,1
897,0
898,1
899,0
900,1


In [20]:
# Submit to competition
from kaggle import api
api.competition_submit_cli('subm.csv', 'autoPyTorch - initialization', comp)

100%|██████████| 2.77k/2.77k [00:00<00:00, 4.99kB/s]


Successfully submitted to Titanic - Machine Learning from Disaster